In [2]:
# Start Run of whole Graph on the GPU 

import pykeen
from pykeen.pipeline import pipeline
import networkx as nx
import pathlib
from random import sample
import pandas as pd
from pykeen.triples import TriplesFactory
import torch
import numpy as np


In [ ]:
##################################
#              NOTES             #
##################################

# sample function is causing the problem with splitting function (ratio)
# same error in default pipeline, but at the beginning not at the end of calculations

# use EGO Graph for finding small subgraph
# prepare checkpoint notebook with embedding accses for Ben


In [6]:
# from https://github.com/nebw/ehrgraphs/blob/master/ehrgraphs/data/data.py#L82-L117
def preprocess_graph_heterogeneous(graph: nx.Graph):
    edge_types = []
    for u, v, data in graph.edges.data():
        edge_types.append(data["edge_type"])

    #edge_codes, edge_types = pd.factorize(edge_types)
    
    node_types = []
    for n, data in graph.nodes.data():
        node_types.append(data["node_type"])

    node_codes, node_types = pd.factorize(node_types)

    preprocessed_graph = nx.DiGraph()
    preprocessed_graph.add_nodes_from(graph.nodes())

    preprocessed_graph.node_codes = node_codes
    preprocessed_graph.node_types = node_types
        
    # drop shortcut edges
    '''exclude_codes = []
    exclude_codes.append(edge_codes[list(edge_types).index("Subsumes")])
    exclude_codes.append(edge_codes[list(edge_types).index("Is a")])
    

    for (u, v, w), c in zip(graph.edges.data("edge_weight"), edge_codes):
        assert w is not None

        # drop shortcut edges
        if c in exclude_codes and w < 1.0:
            continue
        
        preprocessed_graph.add_edge(u, v, edge_weight=w, edge_code=c)
        '''
    # ---- #
    edge_codes, num_counts = np.unique(edge_types, return_counts=True)
    fraction = num_counts / len(edge_types)
    valid_edge_codes = edge_codes[fraction >= 0.01]
    print(len(valid_edge_codes), valid_edge_codes)
    
    for (u,v,w), t in zip(graph.edges.data("edge_weight"), edge_types):
        assert w is not None
        
        # only most relevant relations
        if t in valid_edge_codes and w >=1.0:
            preprocessed_graph.add_edge(u, v, edge_weight=w, edge_code=t)
        
        else : continue
    # ---- #    
    
    preprocessed_graph.edge_types = edge_types

    return preprocessed_graph


In [7]:
#loading the full graph
base_path = pathlib.Path(
    "/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/2_datasets_pre/211110_anewbeginning")
G = nx.readwrite.gpickle.read_gpickle('/data/analysis/ag-reils/ag-reils-shared/cardioRS/data/2_datasets_pre/211110_anewbeginning/graph_full_211122.p')

# building preprocessed ego graph



In [9]:
# -----TEST -----#
SG = preprocess_graph_heterogeneous(G)
print(len(list(SG.edges())))

4 ['Interacts with' 'Is a' 'Is associated with' 'Subsumes']
2383019


In [48]:
print(len(list(G.edges())))

45664725


In [42]:
# ---------- collect the most frequent relations for subgraph splitting ---- #

#SG = preprocess_graph_heterogeneous(G)


tripleList=[]
nodes=[]
for u,v,data in SG.edges.data():
    l=[]
    l.append(u)
    nodes.append(u)
    l.append(data['edge_code'])
    l.append(v)
    nodes.append(v)
    tripleList.append(l)
print('length bevor sampling: ', len(tripleList))
#needs triples as ndarray - shape (n,3), dtype:str 
tripleArray=np.array(sample(tripleList,1000), dtype=str)

length bevor sampling:  2383019


In [10]:
# -------- directly create TRIPLES ------- #

#SG = preprocess_graph_heterogeneous(G)

tripleList=[]
nodes=[]
for u,v,data in SG.edges.data():
    l=[]
    l.append(u)
    nodes.append(u)
    l.append(data['edge_code'])
    l.append(v)
    nodes.append(v)
    tripleList.append(l)

#needs triples as ndarray - shape (n,3), dtype:str 
tripleArray=np.array(tripleList, dtype=str)
print(len(tripleArray))

2383019


In [11]:
# delete NETWROKX Graph
SG.clear()
del SG

In [ ]:
# ----------- directly loading Triples into PyKEEN ------ #

tf2 = TriplesFactory.from_labeled_triples(tripleArray, create_inverse_triples=True)

print(tf2.get_most_frequent_relations(3))
# ----------- Splitting into training, testing, validation ---#

training_factory, testing_factory, validation_factory = tf2.split([0.9 ,0.01])


# ----------- Training without evaluation ------------ #

# Pick a model
from pykeen.models import TransE

kwargs={'triples_factory': training_factory , 'loss': None, 'predict_with_sigmoid':False, 'preferred_device':None, 'random_seed':None}

tf2_model = TransE(**kwargs, embedding_dim=64) # >64, 256

# Pick an optimizer from Torch
from torch.optim import Adam

optimizer = Adam(params=tf2_model.get_grad_params())

# stopper 
from pykeen.stoppers import EarlyStopper
from pykeen.evaluation import RankBasedEvaluator
evaluator=RankBasedEvaluator()

Stopper = EarlyStopper(
        model=tf2_model,
        evaluator=evaluator,
        training_triples_factory=training_factory,
        evaluation_triples_factory=validation_factory,
        #result_tracker=_result_tracker,
        #**stopper_kwargs,
    )

# Pick a training approach (sLCWA or LCWA)
from pykeen.training import SLCWATrainingLoop

training_loop = SLCWATrainingLoop(

    model=tf2_model,

    triples_factory=training_factory,

    optimizer=optimizer,

)
print('used LOSS : ', training_loop.loss)


# Train like Cristiano Ronaldo

_ = training_loop.train(

    triples_factory=training_factory,

    num_epochs=100, # ! ! !

    batch_size=256,
    
    stopper= Stopper, 
    
    num_workers=4
    # result_tracker='wandb'
    # result_tracker_kwargs=
)

print(Stopper.best_epoch, Stopper.best_metric)


using automatically assigned random_state=3971841365


{0, 1, 2}


No cuda devices were available. The model runs on CPU
No random seed is specified. This may lead to non-reproducible results.


used LOSS :  MarginRankingLoss(
  (margin_activation): ReLU()
)


Training epochs on cpu:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/16756 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/16756 [00:00<?, ?batch/s]

Training batches on cpu:   0%|          | 0/16756 [00:00<?, ?batch/s]

In [ ]:
# class RepresentationModule
entity_RepModel = tf2_model.entity_representations[0] # check for more representations

# collect all nodes out of SG2 that contain an edge
nodes = [u for u,v,data in SG2.edges.data()]
for u,v,data in SG2.edges.data():
    nodes.append(v)
print(len(nodes))
#filter out duplicate nodes
nodes = list(dict.fromkeys(nodes))


# make all entities to ids
embedding_dict={}
all_entities = tf2.entities_to_ids(nodes)

embeddings = entity_RepModel(torch.tensor(all_entities, dtype=torch.int)) # cast list elements into tensors

embedding_dict = dict(zip(nodes, embeddings2.detach().numpy() )) #  Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.


In [60]:
#####################
# Easy Pipeline Way #
####################

tf2 = TriplesFactory.from_labeled_triples(tripleArray, create_inverse_triples=True)

print(tf2.get_most_frequent_relations(3))
# ----------- Splitting into training, testing, validation ---#

training_factory, testing_factory, validation_factory = tf2.split([0.9 ,0.01])


# --


results = pipeline(
    training = training_factory,
    testing = testing_factory,
    validation = validation_factory,
    loss='marginranking',
    loss_kwargs=dict(margin=1),
    model = tf2_model,
    epochs = 100, 
    training_loop = 'sLCWA',
    negative_sampler = 'basic',
    evaluator = 'rankbased',
    stopper = 'early',
    stopper_kwargs=dict(frequency=5, patience=2, relative_delta=0.002)
)

results.plot_losses()    

INFO:pykeen.triples.triples_factory:applying cutoff of 3 to TriplesFactory(num_entities=486362, num_relations=6, num_triples=2383019, inverse_triples=True)


{0, 1, 2}


INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [1658357, 23830, 214472]
INFO:pykeen.triples.triples_factory:Creating inverse triples.


Training epochs on cpu:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training batches on cpu:   0%|          | 0/16756 [00:00<?, ?batch/s]

KeyboardInterrupt: 

In [ ]:
# SAVE INTO DATAFRAME 
dsaf
items = embedding_dictitems.items()

data_list = list()

df = pd.DataFrame(data_list)
df.to_csv(index=False)